In [1]:
!pip install pyomo -q
!pip install mistralai -q
!wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00


In [2]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [3]:
API_KEY = ''

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 3
MODEL_TEMPERATURE = 0.7

SYSTEM_PROMPT_1 = """Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."""
SYSTEM_PROMPT_2 = """Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."""
PROBLEM_DESCRIPTION = """A firm that packs refreshments and beers, situated in the province of Valencia (Spain) employs the same syrup to produce its 1.5 l COLI and PEPSA products on its S1 production line. Once processed, each hectolitre of syrup produces 40 units of the 1.5 l COLI product and 20 units of the 1.5 l PEPSA product. If X1 units of the 1.5 l COLI product and X2 units of the 1.5 l PEPSA product are produced, the firm estimates that the daily income obtained in dollars would be given by the following function:
49000 times X1 minus X1 squared plus 30 times X2 minus two times X2 squared.
It costs 150 dollars to buy and process each hectolitre of syrup. The S1 packaging line has a net capacity of producing 7100 1.5 l product units every hour. The firm works 5 days a week in 8h shifts. Given its weekly target coverage, the firm is committed to produce at least half the amount of PEPSA than COLI. Although priority orders tend to amend its production planning, the firm wishes to have a basic product planning that optimises its daily profits."""


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-06-14 13:14:23.014640


## Step 1 - Generate Mathematical Formulation

In [4]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [5]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_1_text = response_1.choices[0].message.content

In [6]:
Markdown(response_1_text)

Parameters:
- Daily income coefficients: a1 = 49000, b1 = -1, c2 = 30, d2 = -2
- Syrup conversion rates: s1 = 40 units/hectolitre, s2 = 20 units/hectolitre
- Syrup processing cost: p = 150 dollars/hectolitre
- Production line capacity: C = 7100 units/hour
- Working hours per day: H = 8 hours/day
- Working days per week: D = 5 days/week

Decision variables:
- X1: Number of 1.5 l COLI product units to be produced
- X2: Number of 1.5 l PEPSA product units to be produced

Objective function:
Maximize daily profit given by the function:

f(X1, X2) = 49000X1 - X1^2 + 30X2 - 2X2^2 - 150 * (X1/s1 + X2/s2)

Constraints:
1. Production capacity constraint:
(X1 + X2) / (H * C) ≤ D

2. Minimum production ratio constraint:
X2 ≥ 0.5 * X1

3. Non-negativity constraint:
X1 ≥ 0, X2 ≥ 0

Now, you can use a mathematical optimization software or a solver to find the optimal values for X1 and X2 that maximize the objective function while satisfying the given constraints.

In [7]:
print(response_1_text)

Parameters:
- Daily income coefficients: a1 = 49000, b1 = -1, c2 = 30, d2 = -2
- Syrup conversion rates: s1 = 40 units/hectolitre, s2 = 20 units/hectolitre
- Syrup processing cost: p = 150 dollars/hectolitre
- Production line capacity: C = 7100 units/hour
- Working hours per day: H = 8 hours/day
- Working days per week: D = 5 days/week

Decision variables:
- X1: Number of 1.5 l COLI product units to be produced
- X2: Number of 1.5 l PEPSA product units to be produced

Objective function:
Maximize daily profit given by the function:

f(X1, X2) = 49000X1 - X1^2 + 30X2 - 2X2^2 - 150 * (X1/s1 + X2/s2)

Constraints:
1. Production capacity constraint:
(X1 + X2) / (H * C) ≤ D

2. Minimum production ratio constraint:
X2 ≥ 0.5 * X1

3. Non-negativity constraint:
X1 ≥ 0, X2 ≥ 0

Now, you can use a mathematical optimization software or a solver to find the optimal values for X1 and X2 that maximize the objective function while satisfying the given constraints.


## Step 2 - Generate the Pyomo Code

In [8]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [9]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [10]:
Markdown(response_2_text)

To solve the optimization problem using Pyomo, we first need to install the necessary packages. If you haven't done so already, you can install Pyomo and a solver (e.g., GLPK) using pip:

```bash
pip install pyomo
pip install pyomo.solvers
pip install glpk
```

Now, let's write the Pyomo code to solve the optimization problem.

```python
import pyomo.environ as pyo

# Sample data
a1 = 49000
b1 = -1
c2 = 30
d2 = -2
s1 = 40
s2 = 20
p = 150
C = 7100
H = 8
D = 5

# Create a ConcreteModel
model = pyo.ConcreteModel()

# Decision variables
model.X1 = pyo.Var(domain=pyo.NonNegativeReals)
model.X2 = pyo.Var(domain=pyo.NonNegativeReals)

# Objective function
def obj_rule(model):
    return a1 * model.X1 + b1 * model.X1**2 + c2 * model.X2 + d2 * model.X2**2 \
           - p * (model.X1 / s1 + model.X2 / s2)

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Constraints
def capacity_constraint_rule(model):
    return (model.X1 + model.X2) <= H * C * D

model.capacity_constraint = pyo.Constraint(rule=capacity_constraint_rule)

def min_production_ratio_constraint_rule(model):
    return model.X2 >= 0.5 * model.X1

model.min_production_ratio_constraint = pyo.Constraint(rule=min_production_ratio_constraint_rule)

# Solve the model
opt = pyo.SolverFactory('glpk')
results = opt.solve(model)

# Print the results
print('Objective value:', pyo.value(model.obj))
print('X1:', pyo.value(model.X1))
print('X2:', pyo.value(model.X2))
```

This code defines the Pyomo model, sets the objective function and constraints, and then solves the model using the GLPK solver. The optimal values for X1 and X2, along with the corresponding objective value, are printed at the end.

In [11]:
print(response_2_text)

To solve the optimization problem using Pyomo, we first need to install the necessary packages. If you haven't done so already, you can install Pyomo and a solver (e.g., GLPK) using pip:

```bash
pip install pyomo
pip install pyomo.solvers
pip install glpk
```

Now, let's write the Pyomo code to solve the optimization problem.

```python
import pyomo.environ as pyo

# Sample data
a1 = 49000
b1 = -1
c2 = 30
d2 = -2
s1 = 40
s2 = 20
p = 150
C = 7100
H = 8
D = 5

# Create a ConcreteModel
model = pyo.ConcreteModel()

# Decision variables
model.X1 = pyo.Var(domain=pyo.NonNegativeReals)
model.X2 = pyo.Var(domain=pyo.NonNegativeReals)

# Objective function
def obj_rule(model):
    return a1 * model.X1 + b1 * model.X1**2 + c2 * model.X2 + d2 * model.X2**2 \
           - p * (model.X1 / s1 + model.X2 / s2)

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Constraints
def capacity_constraint_rule(model):
    return (model.X1 + model.X2) <= H * C * D

model.capacity_constraint = pyo

### Code Executability

In [12]:
import pyomo.environ as pyo

# Sample data
a1 = 49000
b1 = -1
c2 = 30
d2 = -2
s1 = 40
s2 = 20
p = 150
C = 7100
H = 8
D = 5

# Create a ConcreteModel
model = pyo.ConcreteModel()

# Decision variables
model.X1 = pyo.Var(domain=pyo.NonNegativeReals)
model.X2 = pyo.Var(domain=pyo.NonNegativeReals)

# Objective function
def obj_rule(model):
    return a1 * model.X1 + b1 * model.X1**2 + c2 * model.X2 + d2 * model.X2**2 \
           - p * (model.X1 / s1 + model.X2 / s2)

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Constraints
def capacity_constraint_rule(model):
    return (model.X1 + model.X2) <= H * C * D

model.capacity_constraint = pyo.Constraint(rule=capacity_constraint_rule)

def min_production_ratio_constraint_rule(model):
    return model.X2 >= 0.5 * model.X1

model.min_production_ratio_constraint = pyo.Constraint(rule=min_production_ratio_constraint_rule)

# Solve the model
opt = pyo.SolverFactory('ipopt')
results = opt.solve(model)

# Print the results
print('Objective value:', pyo.value(model.obj))
print('X1:', pyo.value(model.X1))
print('X2:', pyo.value(model.X2))

Objective value: 400289176.0419919
X1: 16335.833333340022
X2: 8167.9166666600495


### Solution Correctness